In [1]:
from sqlalchemy import Table, Row, Column, create_engine, MetaData, Integer, String, ForeignKey, Date
from sqlalchemy.orm import DeclarativeBase
import openpyxl
import numpy as np

In [2]:
import os
engine = create_engine("sqlite:///mortality.db", echo=True, pool_pre_ping=True)

assert os.path.isfile("G:/Mortality_rate_vis/mortality.db")

class Base(DeclarativeBase):
    pass

In [3]:
engine

Engine(sqlite:///mortality.db)

In [4]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

In [5]:
class Country(Base):
    __tablename__ = "countrys"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(255))
    code: Mapped[str] = mapped_column(String(255))

    def __repr__(self) -> str:
        return f"Country(id={self.id!r}, name={self.name!r}, code={self.code!r})"
    

class Deaths(Base):
    __tablename__ = "deaths"

    id: Mapped[int] = mapped_column(primary_key=True)
    country_id: Mapped[int] = mapped_column(ForeignKey("countrys.id"))
    year: Mapped[Date] = mapped_column(Date)
    age_group: Mapped[str] = mapped_column(String(255))
    sex: Mapped[str] = mapped_column(String(255))
    number_of_deaths: Mapped[int] = mapped_column()
    death_rate_per_100000: Mapped[int] = mapped_column()

    def __repr__(self):
        return f"Deaths(id={self.id!r}, country_id={self.country_id!r}, year={self.year!r}, age_group={self.age_group!r}, sex={self.sex!r}, deaths_per_100k={self.deaths_per_100k!r})"



In [6]:
Base.metadata

MetaData()

In [7]:
Base.registry

In [8]:
Base.metadata.create_all(engine)

2025-04-02 17:17:27,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-02 17:17:27,996 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("countrys")
2025-04-02 17:17:27,997 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 17:17:27,998 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("deaths")
2025-04-02 17:17:28,000 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 17:17:28,001 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("deaths")
2025-04-02 17:17:28,001 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 17:17:28,003 INFO sqlalchemy.engine.Engine 
CREATE TABLE deaths (
	id INTEGER NOT NULL, 
	country_id INTEGER NOT NULL, 
	year DATE NOT NULL, 
	age_group VARCHAR(255) NOT NULL, 
	sex VARCHAR(255) NOT NULL, 
	number_of_deaths INTEGER NOT NULL, 
	death_rate_per_100000 INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(country_id) REFERENCES countrys (id)
)


2025-04-02 17:17:28,004 INFO sqlalchemy.engine.Engine [no key 0.00056s] ()
2025-04-02 17:17:28,

In [16]:
import pandas as pd

filename = "IHME_GBD_2010_MORTALITY_AGE_SPECIFIC_BY_COUNTRY_1970_2010.csv"

df = pd.read_csv(filename).rename(columns={
    "Country Code" : "code",
    "Country Name" : "name",
    "Year": "year",
    "Age Group" : "age_group",
    "Sex" : "sex",
    "Number of Deaths" : "number_of_deaths",
    "Death Rate Per 100,000" : "death_rate_per_100000"
})

countrys = pd.DataFrame(df[["name", "code"]].drop_duplicates().reset_index(drop=True))


print(countrys)

                     name code
0             Afghanistan  AFG
1                  Angola  AGO
2                 Albania  ALB
3                 Andorra  AND
4    United Arab Emirates  ARE
..                    ...  ...
182                 Samoa  WSM
183                 Yemen  YEM
184          South Africa  ZAF
185                Zambia  ZMB
186              Zimbabwe  ZWE

[187 rows x 2 columns]


In [10]:
# countrys.to_sql(name="countrys", if_exists="append", con=engine, index=True, index_label="id")

In [11]:
deaths = pd.DataFrame(df)

In [17]:
print(df)

      code         name  year  age_group     sex number_of_deaths  \
0      AFG  Afghanistan  1970   0-6 days    Male           19,241   
1      AFG  Afghanistan  1970   0-6 days  Female           12,600   
2      AFG  Afghanistan  1970   0-6 days    Both           31,840   
3      AFG  Afghanistan  1970  7-27 days    Male           15,939   
4      AFG  Afghanistan  1970  7-27 days  Female           11,287   
...    ...          ...   ...        ...     ...              ...   
58900  ZWE     Zimbabwe  2010  80+ years  Female            7,606   
58901  ZWE     Zimbabwe  2010  80+ years    Both           13,305   
58902  ZWE     Zimbabwe  2010   All ages    Male           82,010   
58903  ZWE     Zimbabwe  2010   All ages  Female           77,420   
58904  ZWE     Zimbabwe  2010   All ages    Both          159,430   

      death_rate_per_100000  
0                318,292.90  
1                219,544.20  
2                270,200.70  
3                 92,701.00  
4                 68,

In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:

    for row in 